In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13285,2024-11-04,Brasil Nbb,19:30,Unifacisa,R10 Score Vasco da Gama,1.57,2.20,148.5,1.81,1.85,-4.5,2.01,1.68,jwaLOqAj,0.636943,0.454545,0.552486,0.540541,0.091488,0.4,0.2,NaN,NaN,138.720,72.929676,0.525733,120.640,39.232550,0.325204,129.462,129.592,257.95,102.20,0.0,0.0,0.0,0.0,0.236327,0.015456,0.126474,-1.46,-0.292,-1.952055,0.736855,0.6,-0.136855,0.83,0.166,7.228916,0.515056,0.5,-0.015056
13286,2024-11-04,Brasil Nbb,20:00,Flamengo,Caxias do Sul,1.03,10.50,150.5,1.86,1.81,-20.5,1.96,1.72,WYy8nm2c,0.970874,0.095238,0.537634,0.552486,0.066112,0.6,0.4,NaN,NaN,94.248,22.585733,0.239642,235.270,98.904660,0.420388,90.206,292.078,69.30,365.31,0.0,0.0,0.0,0.0,1.161544,0.019267,0.092231,-0.64,-0.128,-0.234375,0.866245,0.9,0.033755,-0.49,-0.098,-96.938776,0.307250,0.2,-0.107250
13287,2024-11-04,Brasil Nbb,20:00,Franca,São José,1.08,6.75,156.5,1.83,1.83,-13.5,2.00,1.72,vauGpRW9,0.925926,0.148148,0.546448,0.546448,0.074074,0.2,0.6,NaN,NaN,109.820,36.379338,0.331263,204.440,107.910395,0.527834,116.502,223.104,102.30,171.52,0.0,0.0,0.0,0.0,1.024086,0.000000,0.106446,0.56,0.112,0.714286,0.824043,0.9,0.075957,-2.69,-0.538,-10.687732,0.425861,0.2,-0.225861
13288,2024-11-04,Eua Nba,21:00,Cleveland Cavaliers,Milwaukee Bucks,1.35,3.40,227.5,1.84,2.00,-8.5,2.04,1.81,4fvF4IMh,0.740741,0.294118,0.543478,0.500000,0.034858,0.8,0.2,NaN,NaN,196.108,39.519589,0.201520,218.578,154.445393,0.706592,171.888,225.092,156.00,142.56,0.0,0.0,0.0,0.0,0.610345,0.058926,0.084485,-1.07,-0.214,-1.635514,0.585012,0.5,-0.085012,-3.37,-0.674,-3.560831,0.466376,0.1,-0.366376
13289,2024-11-04,Eua Nba,21:15,Washington Wizards,Golden State Warriors,4.14,1.18,231.5,1.80,1.86,7.5,2.01,1.68,0MXN2di5,0.241546,0.847458,0.555556,0.537634,0.089004,0.6,0.8,NaN,NaN,459.460,129.726210,0.282345,231.690,90.001320,0.388456,500.174,244.746,369.46,368.30,1.0,0.0,0.0,0.0,0.786856,0.023184,0.126474,3.06,0.612,5.130719,0.381630,0.4,0.018370,5.02,1.004,0.179283,0.491381,0.7,0.208619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13409,2024-11-04,Eua Ncaa,22:00,Wisconsin,Holy Cross,1.29,3.40,140.5,1.91,1.91,-26.5,2.05,1.71,M5kC3UvN,0.775194,0.294118,0.523560,0.523560,0.069311,0.8,0.6,NaN,NaN,132.758,40.729024,0.306791,283.984,256.633506,0.903690,109.238,360.634,143.55,742.50,0.0,1.0,0.0,0.0,0.636245,0.000000,0.127881,2.07,0.414,0.700483,0.784926,1.0,0.215074,1.62,0.324,7.407407,0.205777,0.2,-0.005777
13410,2024-11-04,Eua Ncaa,22:30,Loyola (Ill).,Chicago St,1.02,18.00,135.5,1.80,1.95,-20.5,2.05,1.71,8jB3mY97,0.980392,0.055556,0.555556,0.512821,0.035948,0.6,0.6,NaN,NaN,130.248,22.223063,0.170621,278.408,143.419092,0.515140,123.428,278.408,161.70,97.35,0.0,0.0,0.0,0.0,1.262531,0.056569,0.127881,3.17,0.634,0.031546,0.691974,0.9,0.208026,2.70,0.540,31.481481,0.228144,0.4,0.171856
13411,2024-11-04,Eua Ncaa,23:00,New Mexico,Nicholls State,1.04,15.00,151.5,1.91,1.91,-17.5,2.00,1.71,YBXzpxYh,0.961538,0.066667,0.523560,0.523560,0.028205,0.4,0.4,NaN,NaN,120.860,26.966631,0.223123,172.394,38.823803,0.225204,100.210,165.260,128.02,190.40,0.0,0.0,0.0,0.0,1.230824,0.000000,0.110545,-0.74,-0.148,-0.270270,0.785147,0.8,0.014853,3.14,0.628,22.292994,0.434436,0.6,0.165564
13412,2024-11-04,Eua Ncaa,23:00,Utah Utes,Alcorn State,1.01,28.00,146.5,1.80,1.95,-23.5,2.05,1.71,xMDQruVq,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Cleveland Cavaliers,Milwaukee Bucks,227.5,1.84,0.8475,Over
1,22:00,Eua Ncaa,Villanova,Lafayette,131.5,1.95,0.7419,Over
